# CORAL Tutorial: Multi-Species Mutation Extraction Pipeline

This notebook provides a comprehensive guide to using CORAL, covering:
- Full pipeline execution (CLI and library)
- Individual component usage
- Plotting and visualization
- PHYLIP integration
- Common workflows

## Table of Contents
1. [Installation & Setup](#installation)
2. [Full Pipeline: CLI Usage](#cli-pipeline)
3. [Full Pipeline: Library Usage](#library-pipeline)
4. [Individual Components](#individual-components)
5. [Plotting & Visualization](#plotting)
6. [PHYLIP Integration](#phylip)
7. [Advanced Workflows](#advanced)


## 1. Installation & Setup {#installation}

First, ensure CORAL is installed and external tools are available.


In [ ]:
# Install CORAL in development mode
# !pip install -e .

# Check installation
import sys
try:
    import coral
    print(f"✓ CORAL imported successfully")
    print(f"  Version: {coral.__version__ if hasattr(coral, '__version__') else 'dev'}")
except ImportError:
    print("✗ CORAL not found. Install with: pip install -e .")
    sys.exit(1)

# Check external tools
import subprocess

tools = ['bwa', 'samtools', 'datasets']
for tool in tools:
    try:
        result = subprocess.run(['which', tool], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✓ {tool} found: {result.stdout.strip()}")
        else:
            print(f"✗ {tool} not found in PATH")
    except:
        print(f"✗ Could not check for {tool}")

print("\nSetup complete!")


## 2. Full Pipeline: CLI Usage {#cli-pipeline}

The easiest way to run CORAL is via the command-line interface. This section shows the main CLI commands.


### 2.1 Single-Species Pipeline (3-species: outgroup + 2 ingroup)

The `run_single` command processes 3 species: one outgroup (reference) and two ingroup species.


In [ ]:
# Example: Drosophila species
# This command downloads genomes, aligns, generates pileup, and extracts mutations

command = """
coral run_single \\
  --outgroup Drosophila_helvetica GCA_963969585.1 \\
  --species Drosophila_pseudoobscura GCF_009870125.1 Drosophila_miranda GCF_003369915.1 \\
  --output ./output_drosophila \\
  --mapq 60 \\
  --suffix MAPQ60 \\
  --cores 4
"""

print("Command:")
print(command)
print("\nKey options:")
print("  --outgroup: Reference species (name and NCBI accession)")
print("  --species: Two ingroup species (4 arguments: name1, acc1, name2, acc2)")
print("  --output: Output directory")
print("  --mapq: Minimum mapping quality (default: 60)")
print("  --suffix: Optional suffix for output directory")
print("  --cores: Number of CPU cores (default: all available)")
print("  --no-cache: Force regeneration of intermediate files")
print("  --quiet: Disable verbose logging")


### 2.2 Multi-Species Pipeline

The `run_multi` command processes multiple species using either a Newick tree or a species list.


In [ ]:
# Option 1: From Newick tree
command_newick = """
coral run_multi \\
  --newick-tree "(((Drosophila_sechellia|GCF_004382195.2,Drosophila_melanogaster|GCF_000001215.4),Drosophila_mauritiana|GCF_004382145.1),Drosophila_santomea|GCF_016746245.2);" \\
  --output ./output_drosophila_multi \\
  --outgroup Drosophila_santomea \\
  --run-id drosophila_multi_run
"""

# Option 2: From species list (JSON format)
command_list = """
coral run_multi \\
  --species-list '[["Drosophila_sechellia","GCF_004382195.2"],["Drosophila_melanogaster","GCF_000001215.4"],["Drosophila_mauritiana","GCF_004382145.1"],["Drosophila_santomea","GCF_016746245.2"]]' \\
  --output ./output_drosophila_multi \\
  --outgroup Drosophila_santomea \\
  --run-id drosophila_multi_run
"""

print("Option 1: Newick tree")
print(command_newick)
print("\nOption 2: Species list")
print(command_list)


### 2.3 Plotting

Generate mutation spectra plots from existing output tables.


In [ ]:
command_plot = """
coral plot \\
  --tables ./output_drosophila/Tables
"""

print("Command:")
print(command_plot)
print("\nThis generates:")
print("  - Normalized mutation spectra plots")
print("  - Coverage plots")
print("  - Mutation density plots")


### 2.4 PHYLIP Integration

Run PHYLIP phylogenetic analysis on mutation matrices.


In [ ]:
command_phylip = """
coral run_phylip \\
  --df ./output_drosophila_multi/matching_bases.csv.gz \\
  --tree ./output_drosophila_multi/annotated_tree.nwk \\
  --mapping ./output_drosophila_multi/species_mapping.json \\
  --phylip-command dnapars \\
  --prefix drosophila_phylip
"""

print("Command:")
print(command_phylip)
print("\nOptions:")
print("  --phylip-command: PHYLIP program (dnapars, dnapenny, etc.)")
print("  --phylip-exe-dir: Path to PHYLIP executables (optional)")
print("  --prefix: Output file prefix")


## 3. Full Pipeline: Library Usage {#library-pipeline}

You can also use CORAL as a Python library for programmatic control.


In [ ]:
from coral import MutationExtractionPipeline, MultiSpeciesMutationPipeline
import os

# Example: Single-species pipeline
pipeline = MutationExtractionPipeline(
    species_list=[
        ("Drosophila_pseudoobscura", "GCF_009870125.1"),
        ("Drosophila_miranda", "GCF_003369915.1")
    ],
    outgroup=("Drosophila_helvetica", "GCA_963969585.1"),
    base_output_dir="./output_library",
    mapq=60,
    suffix="test_run",
    verbose=True,
    cores=4
)

# Run the full pipeline
# pipeline.run()

print("Pipeline initialized!")
print(f"Output directory: {pipeline.output_dir}")
print(f"Run ID: {pipeline.run_id}")


In [ ]:
# Example: Multi-species pipeline
multi_pipeline = MultiSpeciesMutationPipeline(
    species_list=[
        ("Drosophila_sechellia", "GCF_004382195.2"),
        ("Drosophila_melanogaster", "GCF_000001215.4"),
        ("Drosophila_mauritiana", "GCF_004382145.1"),
        ("Drosophila_santomea", "GCF_016746245.2")
    ],
    base_output_dir="./output_multi_library",
    outgroup="Drosophila_santomea",
    run_id="drosophila_multi_lib",
    verbose=True
)

# Run the pipeline
# multi_pipeline.run()

print("Multi-species pipeline initialized!")
print(f"Output directory: {multi_pipeline.output_dir}")
print(f"Run ID: {multi_pipeline.run_id}")


## 4. Individual Components {#individual-components}

You can use individual components for custom workflows. This section shows how to use each component separately.


### 4.1 Genome Download and Management


In [ ]:
from coral.genome_manager import Genome

# Initialize genome object
genome = Genome(
    name="Drosophila_helvetica",
    accession="GCA_963969585.1",
    output_dir="./genomes",
    verbose=True
)

# Download genome (uses NCBI datasets)
# genome.download()

# After download, the genome object has:
# - genome.fasta_path: Path to the FASTA file
# - genome.fastq_path: Path to simulated FASTQ (if generated)
# - genome.output_dir: Output directory

print(f"Genome: {genome.name}")
print(f"Accession: {genome.accession}")
print(f"FASTA path: {genome.fasta_path}")
print(f"Output directory: {genome.output_dir}")


### 4.2 Alignment

The `Aligner` class handles read simulation and alignment to the reference.


In [ ]:
from coral.alignment_manager import Aligner

# Initialize aligner
# Note: You need a reference genome and a target genome
# aligner = Aligner(
#     species="Drosophila_pseudoobscura",
#     reference=genome,  # Reference genome object
#     target_genome=target_genome,  # Target genome object
#     output_dir="./alignments",
#     mapq=60,
#     continuity=True,
#     verbose=True
# )

# Run alignment
# aligner.align()

# After alignment:
# - aligner.final_bam: Path to the final BAM file
# - aligner.species: Species name

print("Aligner class usage:")
print("  - Handles read simulation (sliding window)")
print("  - Aligns reads using BWA (or custom aligner)")
print("  - Filters by MAPQ")
print("  - Supports continuity filtering")


### 4.3 Pileup Generation

Generate multi-taxa pileup from reference and aligned BAMs.


In [ ]:
from coral.pileup_manager import Pileup

# Initialize pileup generator
# pileup = Pileup(
#     outgroup=reference_genome,  # Reference genome object
#     aligners=[aligner1, aligner2],  # List of Aligner objects
#     base_output_dir="./output",
#     run_id="custom_run",
#     verbose=True
# )

# Generate pileup
# pileup.generate()

# Output:
# - pileup.pileup_path: Path to the gzipped pileup file

print("Pileup class usage:")
print("  - Takes reference genome and list of aligners")
print("  - Generates multi-taxa pileup using samtools mpileup")
print("  - Output is gzipped for efficiency")


### 4.4 Mutation Extraction

Extract mutations from pileup files.


In [ ]:
from coral.mutation_extractor_manager import MutationExtractor, FiveMerExtractor, TripletExtractor

# Option 1: Full mutation extractor (triplet mutations)
# extractor = MutationExtractor(
#     reference="Drosophila_helvetica",
#     taxon1="Drosophila_pseudoobscura",
#     taxon2="Drosophila_miranda",
#     pileup_file="./output/run_id.pileup.gz",
#     mutation_output_dir="./output/Mutations",
#     triplet_output_dir="./output/Triplets",
#     verbose=True
# )
# extractor.extract()

# Option 2: 5-mer extractor (for mutation spectra)
# five_mer = FiveMerExtractor(
#     reference="Drosophila_helvetica",
#     taxon1="Drosophila_pseudoobscura",
#     taxon2="Drosophila_miranda",
#     pileup_file="./output/run_id.pileup.gz",
#     output_dir="./output/5mers",
#     verbose=True
# )
# five_mer.extract()

# Option 3: Triplet extractor
# triplet = TripletExtractor(
#     reference="Drosophila_helvetica",
#     taxon1="Drosophila_pseudoobscura",
#     taxon2="Drosophila_miranda",
#     pileup_file="./output/run_id.pileup.gz",
#     output_dir="./output/Triplets",
#     verbose=True
# )
# triplet.extract()

print("Mutation extractors:")
print("  - MutationExtractor: Full triplet mutations (JSON + CSV)")
print("  - FiveMerExtractor: 5-mer context mutations (for spectra)")
print("  - TripletExtractor: Triplet mutations only")


### 4.5 Mutation Normalization

Normalize mutation counts for downstream analysis.


In [ ]:
from coral.mutation_extractor_manager import MutationNormalizer

# Normalize mutation spectra
# normalizer = MutationNormalizer(
#     mutations_dir="./output/Mutations",
#     tables_dir="./output/Tables",
#     verbose=True
# )
# normalizer.normalize()

print("MutationNormalizer:")
print("  - Normalizes mutation counts by context frequency")
print("  - Generates collapsed mutation spectra")
print("  - Outputs TSV files for plotting")


## 5. Plotting & Visualization {#plotting}

CORAL provides several plotting utilities for visualizing mutation patterns.


In [ ]:
from coral.plot_utils import MutationSpectraPlotter, CoveragePlotter, MutationDensityPlotter

# 1. Mutation spectra plotter
# plotter = MutationSpectraPlotter()
# plotter.plot(tables_dir="./output/Tables")

# 2. Coverage plotter
# coverage = CoveragePlotter()
# coverage.plot(pileup_file="./output/run_id.pileup.gz", output_dir="./output/Plots")

# 3. Mutation density plotter
# density = MutationDensityPlotter()
# density.plot(mutations_dir="./output/Mutations", output_dir="./output/Plots")

print("Available plotters:")
print("  - MutationSpectraPlotter: Normalized mutation spectra")
print("  - CoveragePlotter: Coverage across chromosomes")
print("  - MutationDensityPlotter: Mutation density across genome")


## 6. PHYLIP Integration {#phylip}

Run PHYLIP phylogenetic analysis on mutation matrices.


In [ ]:
from coral.run_phylip import run_phylip
import json

# Load species mapping
# with open("./output/species_mapping.json") as f:
#     mapping = json.load(f)

# Run PHYLIP
# run_phylip(
#     command="dnapars",  # or "dnapenny", etc.
#     df_path="./output/matching_bases.csv.gz",
#     tree_path="./output/annotated_tree.nwk",
#     output_dir="./output",
#     prefix="phylip_run",
#     input_string="Y\n",  # PHYLIP interactive input
#     mapping=mapping,
#     phylip_exe_dir=None  # Uses default if None
# )

print("PHYLIP integration:")
print("  - Requires matching_bases.csv.gz from multi-species pipeline")
print("  - Requires annotated_tree.nwk")
print("  - Requires species_mapping.json")
print("  - Supports various PHYLIP commands (dnapars, dnapenny, etc.)")


## 7. Advanced Workflows {#advanced}

### 7.1 Custom Aligner

Use a custom alignment command instead of BWA.


In [ ]:
# Example: Using minimap2 instead of BWA
pipeline_custom = MutationExtractionPipeline(
    species_list=[
        ("Species1", "ACC1"),
        ("Species2", "ACC2")
    ],
    outgroup=("Outgroup", "OUT_ACC"),
    base_output_dir="./output",
    aligner_name="minimap2",
    aligner_cmd="minimap2 -ax map-ont {ref} {reads} | samtools sort -o {bam}",
    verbose=True
)

print("Custom aligner options:")
print("  - aligner_name: Name of aligner (for logging)")
print("  - aligner_cmd: Custom command template")
print("    Use placeholders: {ref}, {reads}, {bam}")


### 7.2 Low MAPQ Filtering

Use different MAPQ thresholds for different analyses.


In [ ]:
# Strict filtering (high MAPQ)
pipeline_strict = MutationExtractionPipeline(
    species_list=[("S1", "A1"), ("S2", "A2")],
    outgroup=("Out", "O1"),
    base_output_dir="./output_strict",
    mapq=60,  # High quality only
    low_mapq=1,
    verbose=True
)

# Relaxed filtering (lower MAPQ)
pipeline_relaxed = MutationExtractionPipeline(
    species_list=[("S1", "A1"), ("S2", "A2")],
    outgroup=("Out", "O1"),
    base_output_dir="./output_relaxed",
    mapq=30,  # Lower threshold
    low_mapq=1,
    verbose=True
)

print("MAPQ filtering:")
print("  - mapq: Minimum MAPQ for alignment filtering")
print("  - low_mapq: Minimum MAPQ for low-quality reads")
print("  - Higher MAPQ = stricter filtering = fewer but higher quality mutations")


### 7.3 Parallel Processing

Control CPU usage for parallel alignment.


In [ ]:
import multiprocessing

# Use all available cores
pipeline_auto = MutationExtractionPipeline(
    species_list=[("S1", "A1"), ("S2", "A2")],
    outgroup=("Out", "O1"),
    base_output_dir="./output",
    cores=None,  # Auto-detect
    verbose=True
)

# Use specific number of cores
pipeline_limited = MutationExtractionPipeline(
    species_list=[("S1", "A1"), ("S2", "A2")],
    outgroup=("Out", "O1"),
    base_output_dir="./output",
    cores=4,  # Use 4 cores
    verbose=True
)

print(f"Available CPU cores: {multiprocessing.cpu_count()}")
print("  - cores=None: Use all available cores")
print("  - cores=N: Use N cores")


### 7.4 Caching and Cleanup

Manage intermediate files and caching behavior.


In [ ]:
from coral.cleanup_manager import PipelineCleaner

# Pipeline with caching (default)
pipeline_cached = MutationExtractionPipeline(
    species_list=[("S1", "A1"), ("S2", "A2")],
    outgroup=("Out", "O1"),
    base_output_dir="./output",
    no_cache=False,  # Use cache (default)
    verbose=True
)

# Pipeline without caching (force regeneration)
pipeline_no_cache = MutationExtractionPipeline(
    species_list=[("S1", "A1"), ("S2", "A2")],
    outgroup=("Out", "O1"),
    base_output_dir="./output",
    no_cache=True,  # Force regeneration
    verbose=True
)

# Manual cleanup
# cleaner = PipelineCleaner(
#     genomes=[genome1, genome2],
#     alignments=[aligner1, aligner2],
#     pileup=pileup_obj,
#     base_dir="./output",
#     verbose=True
# )
# cleaner.clean_intervals()  # Remove interval files
# cleaner.clean_bams()  # Remove BAM files
# cleaner.clean_pileup()  # Remove pileup files

print("Caching:")
print("  - no_cache=False: Skip steps if outputs exist (default)")
print("  - no_cache=True: Force regeneration of all files")
print("\nCleanup:")
print("  - Use PipelineCleaner to remove intermediate files")
print("  - Helps manage disk space")


In [ ]:
# Complete custom workflow example
# Uncomment and modify as needed

"""
from coral.genome_manager import Genome
from coral.alignment_manager import Aligner
from coral.pileup_manager import Pileup
from coral.mutation_extractor_manager import MutationExtractor, MutationNormalizer
from coral.plot_utils import MutationSpectraPlotter

# Step 1: Download genomes
reference = Genome("Drosophila_helvetica", "GCA_963969585.1", "./genomes")
target1 = Genome("Drosophila_pseudoobscura", "GCF_009870125.1", "./genomes")
target2 = Genome("Drosophila_miranda", "GCF_003369915.1", "./genomes")

reference.download()
target1.download()
target2.download()

# Step 2: Align species to reference
aligner1 = Aligner(
    species="Drosophila_pseudoobscura",
    reference=reference,
    target_genome=target1,
    output_dir="./alignments",
    mapq=60,
    verbose=True
)
aligner2 = Aligner(
    species="Drosophila_miranda",
    reference=reference,
    target_genome=target2,
    output_dir="./alignments",
    mapq=60,
    verbose=True
)

aligner1.align()
aligner2.align()

# Step 3: Generate pileup
pileup = Pileup(
    outgroup=reference,
    aligners=[aligner1, aligner2],
    base_output_dir="./output",
    run_id="custom_workflow",
    verbose=True
)
pileup.generate()

# Step 4: Extract mutations
extractor = MutationExtractor(
    reference="Drosophila_helvetica",
    taxon1="Drosophila_pseudoobscura",
    taxon2="Drosophila_miranda",
    pileup_file=pileup.pileup_path,
    mutation_output_dir="./output/Mutations",
    triplet_output_dir="./output/Triplets",
    verbose=True
)
extractor.extract()

# Step 5: Normalize mutations
normalizer = MutationNormalizer(
    mutations_dir="./output/Mutations",
    tables_dir="./output/Tables",
    verbose=True
)
normalizer.normalize()

# Step 6: Generate plots
plotter = MutationSpectraPlotter()
plotter.plot(tables_dir="./output/Tables")
"""

print("Custom workflow steps:")
print("  1. Download genomes")
print("  2. Align species to reference")
print("  3. Generate pileup")
print("  4. Extract mutations")
print("  5. Normalize mutations")
print("  6. Generate plots")


## Summary

This tutorial covered:

1. **CLI Usage**: Running full pipelines via command line
2. **Library Usage**: Programmatic control with Python
3. **Individual Components**: Using each component separately
4. **Plotting**: Visualization tools
5. **PHYLIP**: Phylogenetic analysis integration
6. **Advanced Workflows**: Custom configurations and workflows

For more information, see the [README.md](README.md) file.

### Quick Reference

**CLI Commands:**
- `coral run_single` - 3-species pipeline
- `coral run_multi` - Multi-species pipeline
- `coral plot` - Generate plots
- `coral run_phylip` - PHYLIP analysis

**Key Library Classes:**
- `MutationExtractionPipeline` - Full single pipeline
- `MultiSpeciesMutationPipeline` - Full multi-species pipeline
- `Genome` - Genome download and management
- `Aligner` - Read simulation and alignment
- `Pileup` - Pileup generation
- `MutationExtractor` - Mutation extraction
- `MutationSpectraPlotter` - Plotting utilities

**Common Options:**
- `--mapq` - Mapping quality threshold (default: 60)
- `--cores` - Number of CPU cores
- `--no-cache` - Force regeneration
- `--quiet` - Disable verbose logging
- `--suffix` - Output directory suffix
